In [ ]:
from abc import ABCMeta
from collections import deque
import random
import string

In [ ]:
class Identificacion():
    def __init__(self, valor):
        self.valor = valor
    
    def validar(self):
        return True
        
class Patente(Identificacion):
    def __init__(self, valor):
        super().__init__(valor)
    
    def __init__(self):
        letters = string.ascii_uppercase
        digits = string.digits
        self.valor = random.choice(letters) + random.choice(letters) + random.choice(letters) + random.choice(letters) + \
                        random.choice(digits) + random.choice(digits)
        
    def validar(self):
        if len(self.valor) > 6:
            return False
        for i in range(4):
            if not self.valor[i].isalpha():
                return False
        if not (self.valor[4].isnumeric() and self.valor[4].isnumeric()):
            return False
        return True

In [ ]:
class Vehiculo(metaclass=ABCMeta):
    def __init__(self, tiempo_revision, patente = None):
        self.tiempo_revision = tiempo_revision
        if patente != None:
            if not patente.validar():
                self.patente = None
            else:
                self.patente = patente
        else:
            self.patente = Patente()
        self.tipo = None
    
    def __str__(self):
        return self.tipo + ' con patente ' + str(self.patente.valor)
    
    def encender(self):
        pass

class Auto(Vehiculo):
    def __init__(self):
        super().__init__(5)
        self.tipo = 'Auto'
    
    def encender(self):
        print('[VEHÍCULO] BRRM')
        
class Moto(Vehiculo):
    def __init__(self):
        super().__init__(3)
        self.tipo = 'Moto'
    
    def encender(self):
        print('[VEHÍCULO] RRM RRM')
        
class Bus(Vehiculo):
    def __init__(self):
        super().__init__(10)
        self.tipo = 'Bus'
    
    def encender(self):
        print('[VEHÍCULO] BBBBRRRRRRMMMMM')

In [ ]:
class Mecanico:
    def __init__(self, nombre):
        self.nombre = nombre
        
    def atender(self, vehiculo):
        print('[MECÁNICO] Mecánico ' + self.nombre + ' atendiendo ' + str(vehiculo))
    

In [ ]:
class Planta:
    def __init__(self):
        self.tarea_actual = None
        self.tiempo_revision = 0
        self.cola_revision = deque()
        self.tiempo_espera = []
        self.mecanico = Mecanico('Tygger')
        
    def __call__(self, max_t):
        for t in range(max_t):
            vehiculo = self.llega_vehiculo_nuevo()
            if vehiculo != None:
                self.cola_revision.append(vehiculo)
                print('[COLA] llega {} en tiempo de simulacion t={} min. Hay {} vehiculos en la cola.'.format(self.cola_revision[-1], t, len(self.cola_revision)))
                vehiculo.encender()

            if (self.tarea_actual == None) and (len(self.cola_revision) > 0):
                # se extrae el proximo auto en la cola de atención
                ac_auto = self.cola_revision.popleft()
                self.tiempo_espera.append(ac_auto.tiempo_revision)
                self.proximo_vehiculo(ac_auto)

            # descuenta un tick de tiempo al auto en espera
            self.tick()

        tiempo_promedio = sum(self.tiempo_espera) / len(self.tiempo_espera)
        tiempo_total = sum(self.tiempo_espera)

        print()
        print('Estadísticas:')
        print('Tiempo promedio de espera {0:6.2f} min.'.format(tiempo_promedio))
        print('Tiempo total de atención de la planta fue de {0:6.2f} min'.format(sum(self.tiempo_espera)))
        print('Total de vehículos atendidos: {0}'.format(len(self.tiempo_espera)))
        
    def ocupado(self):
        return self.tarea_actual != None

    def proximo_vehiculo(self, vehiculo):
        self.tarea_actual = vehiculo
        self.tiempo_revision = vehiculo.tiempo_revision
        self.mecanico.atender(vehiculo)
        print('[PLANTA] Atendiendo {0} con un tiempo promedio de {1} min'.format(self.tarea_actual.tipo, self.tiempo_revision))
        
    def tick(self):
        if self.tarea_actual != None:
            self.tiempo_revision -= 1
            if self.tiempo_revision <= 0:
                print('[Planta] termina revision de {}'.format(self.tarea_actual))
                self.tarea_actual = None
    
    def llega_vehiculo_nuevo(self):
        if random.random() >= 0.8:
            v = random.random()
            if v < 0.5:
                return Auto()
            elif v < 0.8:
                return Moto()
            else:
                return Bus()
        else:
            return None

In [ ]:
if __name__ == '__main__':
       
    prt = Planta()
    max_t = 80
    prt(max_t)